# PA 4

This program will build sample classifiers for the pre-processed automobile dataset created for PA1. 

We will use mean value replacement to resolve missing values, as described by the following steps.

## Step 1 - Random Instances: Linear Regression

This step will create a classifier that predicts mpg values using least squares linear regression.

Our predictor attribute will be vehicle weight. The algorithm will take a set of instances, predict their MPG values, and then discretize these values based on the DOE classifications given in PA2.

To test our classifier, we will select 5 random instances from the dataset, and then compare our predicted MPG classification with the actual classification from the data.

To begin, we will first define several helper functions in order to create and clean our dataset.

* `read_data()`
    * Reads a text file for the data to create a dataset from
    * **Parameters**
        * `filename`: The name of the file to read data from
    * **Returns** 
        * A string containing the text from the given file
* `create_dataset()`
    * Turns a formatted string into a 2D dataset array
    * **Parameters**
        * `data`: A string containing the data to build the dataset from
    * **Returns**
        * The data in the dataset as a 2D array
* `resolve_missing_values()`
    * Resolves all instances of "NA" by replacing them with the mean of that attribute, in-place
    * **Parameters**
        * `data`: The dataset to clean

In [1]:
def read_data(filename):
    f = open(filename, 'r')
    text = f.read()
    f.close()
    return text

def create_dataset(data):
    data_r = data.splitlines()
    dataset = []
    for line in data_r:
        instance = line.split(',')
        dataset.append(instance)
    for instance in dataset:
        for i in range(10):
            try:
                instance[i] = float(instance[i])
            except:
                instance[i] = instance[i]
    return dataset

def resolve_missing_values(data):
    for i in range(10):
        if i != 8:
            sum_i = 0
            count_i = 0
            for instance in data:
                if instance[i] != "NA":
                    try:
                        sum_i += instance[i]
                        count_i += 1
                    except:
                        print(instance[i])
            if count_i == 0:
                continue
            mean = sum_i / count_i
            for instance in data:
                if instance[i] == "NA":
                    instance[i] = mean

Now that we have these functions defined, we will call them on the "auto-data.txt" file to create the dataset for this project.

In [2]:
dataset = create_dataset(read_data("auto-data.txt"))
resolve_missing_values(dataset)

Finally, we have a dataset that we can use for the rest of this project.

Now, we will define functions for linear regression. To do this, we will copy the helper functions written for PA3, outlined here for clarity.

* `compute_total_instances()`  
    * **Params**: 
        * `data` = the dataset to query
    * **Returns**: The number of instances in that dataset
* `compute_most_common_element()`
    * **Params**: 
        * `data` = the dataset to query
        * `index` = the index of the attribute to query
    * **Returns**: The most commonly occurring element of that attribute in the dataset
* `sum_attribute()`
    * **Params**:
        * `data` = the dataset to query
        * `index` = the index of the attribute to query
    * **Returns**:
        * The sum of all values for the given index in the dataset
* `mean_attribute()`
    * **Params**:
        * `data` = the dataset to query
        * `index` = the index of the attribute to query
    * **Returns**:
        * The mean of all values of the given attribute in the dataset
* `linear_regression_slope()`
    * **Params**:
        * `data` = the dataset to query
        * `x_index` = the index of the attribute to use for the x values
        * `y_index` = the index of the attribute to use for the y values
    * **Returns**:
        * The slope of the linear regression line
* `lienar_regression_intercept()`
     * **Params**:
        * `data` = the dataset to query
        * `x_index` = the index of the attribute to use for the x values
        * `y_index` = the index of the attribute to use for the y values
    * **Returns**:
        * The intercept of the linear regression line
        
Additionally, we will use these formulas for linear regression:
* **Linear Regression**:  $\overline{y} = m\overline{x} + b$
* **Slope**:  $m = \frac{\sum_{i = 1}^{n} (x_{i}-\overline{x})(y_{i}-\overline{y})}{\sum_{i = 1}^{n} (x_{i}-\overline{x})^{2}}$
* **Intercept**: $\overline{y} - m\overline{x}$

In [3]:
def compute_total_instances(data):
    return len(data)

def compute_most_common_element(data, index):
    freqs = {}
    for instance in data:
        if instance[index] in freqs:
            freqs[instance[index]] += 1
        else:
            freqs[instance[index]] = 1            
    # line referenced from https://stackoverflow.com/questions/268272/getting-key-with-maximum-value-in-dictionary
    mce = max(freqs.items(), key=operator.itemgetter(1))[0]
    return mce

def sum_attribute(data, index):
        sum = 0
        for instance in data:
            sum += instance[index]
        return sum
    
def mean_attribute(data, index):
    mean = sum_attribute(data, index) / compute_total_instances(data)
    return mean
    
    
def linear_regression_slope(data, x_index, y_index):
    mean_x = mean_attribute(data, x_index)
    mean_y = mean_attribute(data, y_index)
    sum_dividend = 0
    sum_divisor = 0
    for i in range(len(data)):
        sum_dividend += (data[i][x_index] - mean_x)*(data[i][y_index] - mean_y)
        sum_divisor += (data[i][x_index] - mean_x)**2
    m = sum_dividend / sum_divisor
    return m

def linear_regression_intercept(data, x_index, y_index):
    mean_x = mean_attribute(data, x_index)
    mean_y = mean_attribute(data, y_index)
    m = linear_regression_slope(data, x_index, y_index)
    b = mean_y - (mean_x * m)
    return b

For the DOE Classifications, we will use these values, defined in PA2:

| Rating | MPG   |
|--------|-----  |
|   10   | ≥ 45  |
|   9    | 37-44 |
|   8    | 31-36 |
|   7    | 27-30 |
|   6    | 24-26 |
|   5    | 20-23 |
|   4    | 17-19 |
|   3    | 15-16 |
|   2    |   14  |
|   1    | ≤ 13  |

To make our predictions, we will define our classifier as follows:
* `mpg_to_DOE()`
    * **Parameters**
        * `mpg`: The actual MPG value of a car instance
    * **Returns**
        * The DOE classification for the given MPG value
* `linear_classifier()`
    * **Parameters**
        * `data`: The dataset to use for defining our linear regression
        * `test`: A list of test instances to make predictions on
    * **Returns**
        * A list of the DOE classification for the predicted MPG values of each test instance

In [4]:
def mpg_to_DOE(mpg):
    if mpg >= 45:
        y = 10
    elif mpg >= 37:
        y = 9
    elif mpg >= 31:
        y = 8
    elif mpg >= 27:
        y = 7
    elif mpg >= 24:
        y = 6
    elif mpg >= 20:
        y = 5
    elif mpg >= 17:
        y = 4
    elif mpg >= 15:
        y = 3
    elif mpg >= 14:
        y = 2
    else:
        y = 1
    return y

def linear_classifier(data, test):
    m = linear_regression_slope(data, 4, 0)
    b = linear_regression_intercept(data, 4, 0)
    predictions = []
    for instance in test:
        x = instance[4]
        mpg = m*x + b
        y = mpg_to_DOE(mpg)
        predictions.append(y)
    return(predictions)

To test this, we will select a random 5 instances from our dataset to predict on, generated using the following helper function:

* `generate_random_instances()`
    * **Parameters**
        * `data`: The dataset to pull random instances from
        * `n`: The number of instances to generate
    * **Returns**
        * A list of _n_ random instances from the dataset

In [5]:
from random import randint
import copy

def generate_random_instances(data, n):
    data_c = copy.deepcopy(data)
    test_instances = []
    for i in range(n):
        index = randint(0, len(data_c)-1)
        instance = data_c.pop(index)
        test_instances.append(instance)
    return test_instances

The following cell will run the classifier function on the test instances and display the output.

In [6]:
test_instances = generate_random_instances(dataset, 5)
predictions_1 = linear_classifier(dataset, test_instances)

def print_output_1(test_instances, predictions):
    print("===========================================")
    print("STEP 1: Linear Regression MPG Classifier")
    print("===========================================")
    for i in range(len(predictions)):
        print("instance: ", end="")
        test = copy.deepcopy(test_instances[i])
        mpg = test.pop(0)
        actual = mpg_to_DOE(mpg)
        instance_string = ""
        for attribute in test:
            instance_string += str(attribute)
            instance_string += ", "
        instance_string.rstrip(", ")
        print(instance_string)
        print("class:", predictions[i], end="")
        print(", actual:", actual)

print_output_1(test_instances, predictions_1)

## Step 2 - Random Instances: kNN

For this step, we will instead create a nearest neighbor classifier for mpg instead of linear regression.

To do this, we will need several helper functions for computing kNN:

* `normalize_instance()`
    * **Parameters**
        * `data`: The dataset to use for calculating the min and max values for normalization
        * `instance`: The instance of the dataset to normalize
        * `class_index`: The index of the classification attribute, which should not be normalized
    * **Returns**
        * The given instance, normalized
* `calculate_distance()`
    * **Parameters**
        * `instance`: The instance to calculate the distance to
        * `base`: The instance to calculate the distance from
    * **Returns**
        * The distance between the two instances, using Euclidean Distance Formula
* `calculate_all_distances()`
    * **Parameters**
        * `data`: The dataset to calculate distances against
        * `base`: The instance to calculate distances for
        * `class_index`: The index of the classification attribute, which is not included in the distance formula
    * **Returns**
        * A list of all distances to the base instance, along with their corresponding indices in the dataset
* `knn()`
    * **Parameters**
        * `data`: The dataset to use for knn calculations
        * `instance`: The instance to classify
        * `k`: The k value for kNN (how many neighbors to use for prediction)
        * `class_index`: The index of the classification attribute, which is not included in the distance formula
    * **Returns**
        * The predicted classification of `instance`, based on kNN.

In [7]:
import operator

def normalize_instance(data, instance, class_index):
    normalized_instance = []
    for i in range(len(instance)):
        if i == class_index:
            normalized_instance.append(instance[i])
            continue
        xs = []
        for line in data:
            xs.append(line[i])
        x = instance[i]
        normal = (x - min(xs)) / (max(xs) - min(xs))
        normalized_instance.append(normal)
    return normalized_instance

def calculate_distance(instance, base):
    sum_a = 0
    for i in range(len(instance)):
        dif = instance[i] - base[i]
        sum_a += dif**2
    distance = sum_a**(1/2)
    return distance

def calculate_all_distances(data, base, class_index):
    distances = []
    for i in range(len(data)):
        instance = copy.deepcopy(data[i])
        instance.pop(class_index)
        distance = calculate_distance(instance, base)
        distances.append((i, distance))
    return distances

def knn(data, instance, k, class_index):
    instance = normalize_instance(data, instance, class_index)
    data_n = []
    for line in data:
        line_n = normalize_instance(data, line, class_index)
        data_n.append(line_n)
    distances = calculate_all_distances(data_n, instance, class_index)
    sorted_distances = sorted(distances, key=operator.itemgetter(1))
    neighbor_indices = []
    for i in range(k):
        neighbor_indices.append(sorted_distances[i][0])
    neighbor_classes = []
    for i in neighbor_indices:
        neighbor_classes.append(data[i][class_index])
    class_list = [[x] for x in neighbor_classes]
    return compute_most_common_element(class_list, 0)

Now we will create our classifier in a similar way to Step 1. First, we will create a curtailed dataset that only contains the attributes we will use for distance calculations, which simplifies our logic.

In [8]:
def create_step2_data(data):
    data_o = []
    for instance in data:
        instance_o = []
        instance_o.append(instance[1])
        instance_o.append(instance[4])
        instance_o.append(instance[5])
        instance_o.append(mpg_to_DOE(instance[0]))
        data_o.append(instance_o)
    return data_o

def knn_classifier(data, test, k, class_index):
    predictions = []
    for instance in test:
        predictions.append(knn(data, instance, k, class_index))
    return predictions



Finally, we can generate an output that runs our classifier on a set of 5 test instances.

In [9]:
data_2 = create_step2_data(dataset)
knn_test_instances_raw = generate_random_instances(dataset, 5)
knn_test_instances = create_step2_data(knn_test_instances_raw)
predictions_2 = knn_classifier(data_2, knn_test_instances, 5, 3)

def print_output_2(test_instances, predictions):
    print("===========================================")
    print("STEP 2: k=5 Nearest Neighbor MPG Classifier")
    print("===========================================")
    for i in range(len(predictions)):
        print("instance: ", end="")
        test = copy.deepcopy(test_instances[i])
        mpg = test.pop(0)
        actual = mpg_to_DOE(mpg)
        instance_string = ""
        for attribute in test:
            instance_string += str(attribute)
            instance_string += ", "
        instance_string.rstrip(", ")
        print(instance_string)
        print("class:", predictions[i], end="")
        print(", actual:", actual)
        
frprint_output_2(knn_test_instances_raw, predictions_2)

STEP 2: k=5 Nearest Neighbor MPG Classifier
instance: 8.0, 360.0, 215.0, 4615.0, 14.0, 70.0, 1.0, "ford f250", 3214.0, 
class: 1, actual: 1
instance: 6.0, 232.0, 100.0, 2914.0, 16.0, 75.0, 1.0, "amc gremlin", 2798.0, 
class: 5, actual: 5
instance: 4.0, 90.0, 70.0, 1937.0, 14.2, 76.0, 2.0, "vw rabbit", 3499.0, 
class: 7, actual: 7
instance: 4.0, 113.0, 95.0, 2278.0, 15.5, 72.0, 3.0, "toyota corona hardtop", 2532.0, 
class: 6, actual: 6
instance: 6.0, 168.0, 120.0, 3820.0, 16.7, 76.0, 2.0, "mercedes-benz 280s", 17124.0, 
class: 3, actual: 3


In [74]:
from random import shuffle

def compute_accuracy(predictions, actual):
    correct = 0
    for i in range(len(predictions)):
        if predictions[i] == actual[i]:
            correct += 1
    accuracy = correct / len(predictions)
    return accuracy

def compute_error(predictions, actual):
    incorrect = 0
    for i in range(len(predictions)):
        if predictions[i] != actual[i]:
            incorrect += 1
    error = incorrect / len(predictions)
    return error

def compute_linear(test, train):
    predictions = linear_classifier(train, test)
    actual = [mpg_to_DOE(instance[0]) for instance in test]
    accuracy = compute_accuracy(predictions, actual)
    error_rate = compute_error(predictions, actual)
    return accuracy, error_rate

def compute_knn(test, train, k):
    data = create_step2_data(train)
    test_instances = create_step2_data(test)
    predictions = knn_classifier(data, test_instances, k, 3)
    actual = [instance[3] for instance in test_instances]
    accuracy = compute_accuracy(predictions, actual)
    error_rate = compute_error(predictions, actual)
    return accuracy, error_rate

In [75]:
def create_random_subsample(data, size):
    cutoff = int(len(data) * size)
    data_c = copy.deepcopy(data)
    shuffle(data_c)
    train = data_c[:cutoff]
    test = data_c[cutoff + 1:]
    return test, train
    
def print_output_3(data, k, train_size, test_size):
    size = (test_size) / (test_size + train_size)
    sum_accuracy_l = 0
    sum_accuracy_k = 0
    sum_error_l = 0
    sum_error_k = 0
    for i in range(k):
        test, train = create_random_subsample(data, size)
        accuracy_l, error_l = compute_linear(test, train)
        accuracy_k, error_k = compute_knn(test, train, 5)
        sum_accuracy_l += accuracy_l
        sum_error_l += error_l
        sum_accuracy_k += accuracy_k
        sum_error_k += error_k
    mean_accuracy_l = sum_accuracy_l / k
    mean_accuracy_k = sum_accuracy_k / k
    mean_error_l = sum_error_l / k
    mean_error_k = sum_error_k / k
    print("===========================================")
    print("STEP 3: Predictive Accuracy")
    print("===========================================")
    print("Random Subsample (k=", k, " ", train_size, ":", test_size, " Train/Test)", sep="")
    print("Linear Regression: accuracy = %.2f, error rate = %.2f" %(mean_accuracy_l, mean_error_l))
    print("k Nearest Neighbor: accuracy = %.2f, error rate = %.2f" %(mean_accuracy_k, mean_error_k))
    
    
print_output_3(dataset, 10, 2, 1)

STEP 3: Predictive Accuracy
Random Subsample (k=10 2:1 Train/Test)
Linear Regression: accuracy = 0.42, error rate = 0.58
k Nearest Neighbor: accuracy = 0.37, error rate = 0.63


In [76]:
def create_cross_fold(data, n):
    data_r = copy.deepcopy(data)
    shuffle(data_r)
    size = int(len(data) * 1/n) 
    start = 0
    end = size
    folds = []
    for i in range(n-1):
        folds.append(data[start:end])
        start = end + 1
        end += size + 1
    folds.append(data[start:])
    return folds

def print_output_4(data, n):
    sum_accuracy_l = 0
    sum_accuracy_k = 0
    sum_error_l = 0
    sum_error_k = 0
    folds = create_cross_fold(data, n)
    for i in range(n):
        test = folds[i]
        train = []
        for x in range(n):
            if x == i:
                continue
            train += folds[x]
        accuracy_l, error_l = compute_linear(test, train)
        accuracy_k, error_k = compute_knn(test, train, 5)
        sum_accuracy_l += accuracy_l
        sum_error_l += error_l
        sum_accuracy_k += accuracy_k
        sum_error_k += error_k
    mean_accuracy_l = sum_accuracy_l / n
    mean_accuracy_k = sum_accuracy_k / n
    mean_error_l = sum_error_l / n
    mean_error_k = sum_error_k / n
    print("===========================================")
    print("STEP 4: Predictive Accuracy")
    print("===========================================")
    print("Stratified ", n, "-Fold Cross Validation", sep="")
    print("Linear Regression: accuracy = %.2f, error rate = %.2f" %(mean_accuracy_l, mean_error_l))
    print("k Nearest Neighbor: accuracy = %.2f, error rate = %.2f" %(mean_accuracy_k, mean_error_k))
    
print_output_4(dataset, 10)

STEP 4: Predictive Accuracy
Stratified 10-Fold Cross Validation
Linear Regression: accuracy = 0.41, error rate = 0.59
k Nearest Neighbor: accuracy = 0.34, error rate = 0.66


In [89]:
from tabulate import tabulate

def confusion_matrix_linear(data, n):
    results = []
    for i in range(10):
        row = [i + 1]
        for i in range(10):
            row.append(0)
        results.append(row)
    folds = create_cross_fold(data, n)
    for i in range(n):
        test = folds[i]
        train = []
        for x in range(n):
            if x == i:
                continue
            train += folds[x]
        predictions = linear_classifier(train, test)
        actual = [mpg_to_DOE(instance[0]) for instance in test]
        for i in range(len(actual)):
            results[actual[i]-1][predictions[i]] += 1
    for row in results:
        row.append(sum(row) - row[0])
        try:
            recognition = (row[row[0]] / row[11])*100
        except ZeroDivisionError:
            recognition = 0
        finally:
            row.append(recognition)
    header = ["MPG"]
    for i in range(1, 11):
        header.append(str(i))
    header += ["Total", "Recognition (%)"]
    print("Linear Regression (Stratefied", n, "-Fold Cross Validation Results):", sep="")
    print("===============================================================")
    print(tabulate(results, headers=header))
    
confusion_matrix_linear(dataset, 10)

Linear Regression (Stratefied10-Fold Cross Validation Results):
  MPG    1    2    3    4    5    6    7    8    9    10    Total    Recognition (%)
-----  ---  ---  ---  ---  ---  ---  ---  ---  ---  ----  -------  -----------------
    1   22    1    6    2    1    0    0    0    0     0       32            68.75
    2    9    5    3    2    1    0    0    0    0     0       20            25
    3   10    5    8   13    2    0    0    0    0     0       38            21.0526
    4    0    1    8   21   21    3    0    0    0     0       54            38.8889
    5    0    2    3   10   28   19    1    0    0     0       63            44.4444
    6    0    0    0    1    7   25    8    0    0     0       41            60.9756
    7    0    0    0    0    3   12   18    0    0     0       33            54.5455
    8    0    0    0    0    0    2   20    0    0     0       22             0
    9    0    0    0    0    0    1    2    0    0     0        3             0
   10    0    0   

In [92]:
def confusion_matrix_knn(data, n, k):
    results = []
    for i in range(10):
        row = [i + 1]
        for i in range(10):
            row.append(0)
        results.append(row)
    folds = create_cross_fold(data, n)
    for i in range(n):
        test = folds[i]
        train = []
        for x in range(n):
            if x == i:
                continue
            train += folds[x]
        data = create_step2_data(train)
        test_instances = create_step2_data(test)
        predictions = knn_classifier(data, test_instances, k, 3)
        actual = [instance[3] for instance in test_instances]
        for i in range(len(actual)):
            results[actual[i]-1][predictions[i]] += 1
    for row in results:
        row.append(sum(row) - row[0])
        try:
            recognition = (row[row[0]] / row[11])*100
        except ZeroDivisionError:
            recognition = 0
        finally:
            row.append(recognition)
    header = ["MPG"]
    for i in range(1, 11):
        header.append(str(i))
    header += ["Total", "Recognition (%)"]
    print("k-NN (Stratefied", n, "-Fold Cross Validation Results):", sep="")
    print("========================================================")
    print(tabulate(results, headers=header))
    
confusion_matrix_knn(dataset, 10, 5)

k-NN (Stratefied10-Fold Cross Validation Results):
  MPG    1    2    3    4    5    6    7    8    9    10    Total    Recognition (%)
-----  ---  ---  ---  ---  ---  ---  ---  ---  ---  ----  -------  -----------------
    1   14    6    6    4    2    0    0    0    0     0       32            43.75
    2    4    1   14    1    0    0    0    0    0     0       20             5
    3   10    6    6   11    5    0    0    0    0     0       38            15.7895
    4    8    2    7   19   16    2    0    0    0     0       54            35.1852
    5    4    0    2   20   20   15    1    1    0     0       63            31.746
    6    0    0    0    2    9   21    7    2    0     0       41            51.2195
    7    0    0    0    1    1   11   11    9    0     0       33            33.3333
    8    0    0    0    0    0    2    8   12    0     0       22            54.5455
    9    0    0    0    0    0    1    1    1    0     0        3             0
   10    0    0    0    0  